<a href="https://colab.research.google.com/github/PazMatias/TpAndroid/blob/master/HPC/Cuaderno_2_Miercoles_grupo13_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduccion 

Como punto de investigacion sobre HPC decidimos profundizar sobre el procesamiento de lenguaje natural, especificamente haciendo uso de la biblioteca spaCy. Mediante el uso de la misma podemos analizar un texto sintacticamente disponibilizando los idiomas ingles, español y aleman. Cabe aclarar que la biblioteca tiene suporte para mas de 60 idiomas disponibles en la pagina web de la misma. Nuestro ejemplo practico tiene como objetivo probar el comportamiento del analisis de los textos y su rendimiento a medida que los mismos varian en volumen de datos, haciendo la comparación directa entre la ejecución de la operación realizada con CPU y GPU. 


Ejemplos propuestos:
- https://github.com/PazMatias/TpAndroid/blob/master/HPC/texto_largo.txt
- https://github.com/PazMatias/TpAndroid/blob/master/HPC/texto_corto.txt

# Armado del Ambiente

Aqui instalaremos la biblioteca spaCy junto con los diferentes paquetes de idiomas a utilizar.

In [ ]:
!pip install -U spacy
!pip install --upgrade spacy
!python -m spacy download en
!python -m spacy download de
!python -m spacy download es
!pip install -U spacy[cuda92]

     |████████████████████████████████| 6.0 MB 12.4 MB/s 
     |████████████████████████████████| 628 kB 40.8 MB/s 
     |████████████████████████████████| 451 kB 63.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 10.1 MB 42.9 MB/s 
     |████████████████████████████████| 181 kB 22.1 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
⚠ As of spaCy v3.0, shortcuts like 'en' are d

# Desarrollo en CPU

In [1]:
 %matplotlib inline
import spacy
from datetime import datetime
import matplotlib.pyplot as plt


# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------

texto_a_analizar="It was the middle of winter, and the snow-flakes were falling like feathers from the sky, and a queen sat at her window working, and her embroidery-frame was of ebony. And as she worked, gazing at times out on the snow, she pricked her finger, and there fell from it three drops of blood on the snow. And when she saw how bright and red it looked, she said to herself, \"Oh that I had a child as white as snow, as red as blood, and as black as the wood of the embroidery frame!\" Not very long after she had a daughter, with a skin as white as snow, lips as red as blood, and hair as black as ebony, and she was named Snow-white. And when she was born the queen died. After a year had gone by the king took another wife, a beautiful woman, but proud and overbearing, and she could not bear to be surpassed in beauty by any one. She had a magic looking-glass, and she used to stand before it, and look in it, and say,  \"Looking-glass upon the wall, Who is fairest of us all?\"  And the looking-glass would answer,  \"You are fairest of them all.\"  And she was contented, for she knew that the looking-glass spoke the truth. Now, Snow-white was growing prettier and prettier, and when she was seven years old she was as beautiful as day, far more so than the queen herself. So one day when the queen went to her mirror and said,  \"Looking-glass upon the wall, Who is fairest of us all?\"  It answered,  \"Queen, you are full fair, 'tis true, But Snow-white fairer is than you.\"  This gave the queen a great shock, and she became yellow and green with envy, and from that hour her heart turned against Snow-white, and she hated her. And envy and pride like ill weeds grew in her heart higher every day, until she had no peace day or night. At last she sent for a huntsman, and said, \"Take the child out into the woods, so that I may set eyes on her no more. You must put her to death, and bring me her heart for a token.\" The huntsman consented, and led her away; but when he drew his cutlass to pierce Snow-white's innocent heart, she began to weep, and to say, \"Oh, dear huntsman, do not take my life; I will go away into the wild wood, and never come home again.\" And as she was so lovely the huntsman had pity on her, and said, \"Away with you then, poor child;\" for he thought the wild animals would be sure to devour her, and it was as if a stone had been rolled away from his heart when he spared to put her to death. Just at that moment a young wild boar came running by, so he caught and killed it, and taking out its heart, he brought it to the queen for a token. And it was salted and cooked, and the wicked woman ate it up, thinking that there was an end of Snow-white.  Now, when the poor child found herself quite alone in the wild woods, she felt full of terror, even of the very leaves on the trees, and she did not know what to do for fright. Then she began to run over the sharp stones and through the thorn bushes, and the wild beasts after her, but they did her no harm. She ran as long as her feet would carry her; and when the evening drew near she came to a little house, and she went inside to rest. Everything there was very small, but as pretty and clean as possible. There stood the little table ready laid, and covered with a white cloth, and seven little plates, and seven knives and forks, and drinking-cups. By the wall stood seven little beds, side by side, covered with clean white quilts. Snow-white, being very hungry and thirsty, ate from each plate a little porridge and bread, and drank out of each little cup a drop of wine, so as not to finish up one portion alone. After that she felt so tired that she lay down on one of the beds, but it did not seem to suit her; one was too long, another too short, but at last the seventh was quite right; and so she lay down upon it, committed herself to heaven, and fell asleep.  When it was quite dark, the masters of the house came home. They were seven dwarfs, whose occupation was to dig underground among the mountains. When they had lighted their seven candles, and it was quite light in the little house, they saw that some one must have been in, as everything was not in the same order in which they left it. The first said, \"Who has been sitting in my little chair?\" The second said, \"Who has been eating from my little plate?\" The third said, \"Who has been taking my little loaf?\" The fourth said, \"Who has been tasting my porridge?\" The fifth said, \"Who has been using my little fork?\" The sixth said, \"Who has been cutting with my little knife?\" The seventh said, \"Who has been drinking from my little cup?\" Then the first one, looking round, saw a hollow in his bed, and cried, \"Who has been lying on my bed?\" And the others came running, and cried, \"Some one has been on our beds too!\" But when the seventh looked at his bed, he saw little Snow-white lying there asleep. Then he told the others, who came running up, crying out in their astonishment, and holding up their seven little candles to throw a light upon Snow-white. \"O goodness! O gracious!\" cried they, \"what beautiful child is this?\" and were so full of joy to see her that they did not wake her, but let her sleep on. And the seventh dwarf slept with his comrades, an hour at a time with each, until the night had passed. When it was morning, and Snow-white awoke and saw the seven dwarfs, she was very frightened; but they seemed quite friendly, and asked her what her name was, and she told them; and then they asked how she came to be in their house. And she related to them how her step-mother had wished her to be put to death, and how the huntsman had spared her life, and how she had run the whole day long, until at last she had found their little house. Then the dwarfs said, \"If you will keep our house for us, and cook, and wash, and make the beds, and sew and knit, and keep everything tidy and clean, you may stay with us, and you shall lack nothing.\" - \"With all my heart,\" said Snow-white; and so she stayed, and kept the house in good order. In the morning the dwarfs went to the mountain to dig for gold; in the evening they came home, and their supper had to be ready for them. All the day long the maiden was left alone, and the good little dwarfs warned her, saying, \"Beware of your step-mother, she will soon know you are here. Let no one into the house.\" Now the queen, having eaten Snow-white's heart, as she supposed, felt quite sure that now she was the first and fairest, and so she came to her mirror, and said,  \"Looking-glass upon the wall, Who is fairest of us all?\"  And the glass answered,  \"Queen, thou art of beauty rare, But Snow-white living in the glen With the seven little men Is a thousand times more fair.\"  Then she was very angry, for the glass always spoke the truth, and she knew that the huntsman must have deceived her, and that Snow-white must still be living. And she thought and thought how she could manage to make an end of her, for as long as she was not the fairest in the land, envy left her no rest. At last she thought of a plan; she painted her face and dressed herself like an old pedlar woman, so that no one would have known her. In this disguise she went across the seven mountains, until she came to the house of the seven little dwarfs, and she knocked at the door and cried, \"Fine wares to sell! fine wares to sell!\" Snow-white peeped out of the window and cried, \"Good-day, good woman, what have you to sell?\" - \"Good wares, fine wares,\" answered she, \"laces of all colours;\"and she held up a piece that was woven of variegated silk. \"I need not be afraid of letting in this good woman,\" thought Snow-white, and she unbarred the door and bought the pretty lace. \"What a figure you are, child!\" said the old woman, \"come and let me lace you properly for once.\" Snow-white, suspecting nothing, stood up before her, and let her lace her with the new lace; but the old woman laced so quick and tight that it took Snow-white's breath away, and she fell down as dead. \"Now you have done with being the fairest,\" said the old woman as she hastened away. Not long after that, towards evening, the seven dwarfs came home, and were terrified to see their dear Snow-white lying on the ground, without life or motion; they raised her up, and when they saw how tightly she was laced they cut the lace in two; then she began to draw breath, and little by little she returned to life. When the dwarfs heard what had happened they said, \"The old pedlar woman was no other than the wicked queen; you must beware of letting any one in when we are not here!\" And when the wicked woman got home she went to her glass and said,  \"Looking-glass against the wall, Who is fairest of us all?\"  And it answered as before,  \"Queen, thou art of beauty rare, But Snow-white living in the glen With the seven little men Is a thousand times more fair.\"  When she heard that she was so struck with surprise that all the blood left her heart, for she knew that Snow-white must still be living. \"But now,\" said she, \"I will think of something that will be her ruin.\" And by witchcraft she made a poisoned comb. Then she dressed herself up to look like another different sort of old woman. So she went across the seven mountains and came to the house of the seven dwarfs, and knocked at the door and cried, \"Good wares to sell! good wares to sell!\" Snow-white looked out and said, \"Go away, I must not let anybody in.\" - \"But you are not forbidden to look,\" said the old woman, taking out the poisoned comb and holding it up. It pleased the poor child so much that she was tempted to open the door; and when the bargain was made the old woman said, \"Now, for once your hair shall be properly combed.\" Poor Snow-white, thinking no harm, let the old woman do as she would, but no sooner was the comb put in her hair than the poison began to work, and the poor girl fell down senseless. \"Now, you paragon of beauty,\" said the wicked woman, \"this is the end of you,\" and went off. By good luck it was now near evening, and the seven little dwarfs came home. When they saw Snow-white lying on the ground as dead, they thought directly that it was the step-mother's doing, and looked about, found the poisoned comb, and no sooner had they drawn it out of her hair than Snow-white came to herself, and related all that had passed. Then they warned her once more to be on her guard, and never again to let any one in at the door. And the queen went home and stood before the looking-glass and said,  \"Looking-glass against the wall, Who is fairest of us all?\"  And the looking-glass answered as before,  \"Queen, thou art of beauty rare, But Snow-white living in the glen With the seven little men Is a thousand times more fair.\"  When she heard the looking-glass speak thus she trembled and shook with anger. \"Snow-white shall die,\" cried she, \"though it should cost me my own life!\" And then she went to a secret lonely chamber, where no one was likely to come, and there she made a poisonous apple. It was beautiful to look upon, being white with red cheeks, so that any one who should see it must long for it, but whoever ate even a little bit of it must die. When the apple was ready she painted her face and clothed herself like a peasant woman, and went across the seven mountains to where the seven dwarfs lived. And when she knocked at the door Snow-white put her head out of the window and said, \"I dare not let anybody in; the seven dwarfs told me not.\" - \"All right,\" answered the woman; \"I can easily get rid of my apples elsewhere. There, I will give you one.\" - \"No,\" answered Snow-white, \"I dare not take anything.\" - \"Are you afraid of poison?\" said the woman, \"look here, I will cut the apple in two pieces; you shall have the red side, I will have the white one.\" For the apple was so cunningly made, that all the poison was in the rosy half of it. Snow-white longed for the beautiful apple, and as she saw the peasant woman eating a piece of it she could no longer refrain, but stretched out her hand and took the poisoned half. But no sooner had she taken a morsel of it into her mouth than she fell to the earth as dead. And the queen, casting on her a terrible glance, laughed aloud and cried, \"As white as snow, as red as blood, as black as ebony! this time the dwarfs will not be able to bring you to life again.\" And when she went home and asked the looking-glass,  \"Looking-glass against the wall, Who is fairest of us all?\"  at last it answered,  \"You are the fairest now of all.\"  Then her envious heart had peace, as much as an envious heart can have. The dwarfs, when they came home in the evening, found Snow-white lying on the ground, and there came no breath out of her mouth, and she was dead. They lifted her up, sought if anything poisonous was to be found, cut her laces, combed her hair, washed her with water and wine, but all was of no avail, the poor child was dead, and remained dead. Then they laid her on a bier, and sat all seven of them round it, and wept and lamented three whole days. And then they would have buried her, but that she looked still as if she were living, with her beautiful blooming cheeks. So they said, \"We cannot hide her away in the black ground.\" And they had made a coffin of clear glass, so as to be looked into from all sides, and they laid her in it, and wrote in golden letters upon it her name, and that she was a king's daughter. Then they set the coffin out upon the mountain, and one of them always remained by it to watch. And the birds came too, and mourned for Snow-white, first an owl, then a raven, and lastly, a dove. Now, for a long while Snow-white lay in the coffin and never changed, but looked as if she were asleep, for she was still as' white as snow, as red as blood, and her hair was as black as ebony. It happened, however, that one day a king's son rode through the wood and up to the dwarfs' house, which was near it. He saw on the mountain the coffin, and beautiful Snow-white within it, and he read what was written in golden letters upon it. Then he said to the dwarfs, \"Let me have the coffin, and I will give you whatever you like to ask for it.\" But the dwarfs told him that they could not part with it for all the gold in the world. But he said, \"I beseech you to give it me, for I cannot live without looking upon Snow-white; if you consent I will bring you to great honour, and care for you as if you were my brethren.\" When he so spoke the good little dwarfs had pity upon him and gave him the coffin, and the king's son called his servants and bid them carry it away on their shoulders. Now it happened that as they were going along they stumbled over a bush, and with the shaking the bit of poisoned apple flew out of her throat. It was not long before she opened her eyes, threw up the cover of the coffin, and sat up, alive and well. \"Oh dear! where am I?\" cried she. The king's son answered, full of joy, \"You are near me,\" and, relating all that had happened, he said, \"I would rather have you than anything in the world; come with me to my father's castle and you shall be my bride.\" And Snow-white was kind, and went with him, and their wedding was held with pomp and great splendour. But Snow-white's wicked step-mother was also bidden to the feast, and when she had dressed herself in beautiful clothes she went to her looking-glass and said,  \"Looking-glass upon the wall, Who is fairest of us all?\"  The looking-glass answered,  ''O Queen, although you are of beauty rare, The young bride is a thousand times more fair.\"" #@param {type:"string"}
idioma = "Ingles" #@param ["Aleman", "Espanol", "Ingles"]

idioma_seleccionado ={

'Aleman' : "de_core_web_sm",

'Espanol' : "es_core_news_sm",

'Ingles' : "en_core_web_sm",


}

if texto_a_analizar == "":
  print("No se ingreso ningun texto a analizar.")
else:
  tiempo_total = datetime.now()

# Se tokeniza el paquete de idioma seleccionado.

  nlp = spacy.load(idioma_seleccionado.get(idioma,-1))

# Procesamos el texto ingresado
  tiempo_text = datetime.now()
  doc = nlp(texto_a_analizar)
  tiempo_text = datetime.now() - tiempo_text

# Analizamos la sintaxis
  print("Cantidad de Sustantivos: ", sum(1 for _ in doc.noun_chunks))
  print("Cantidad de Verbos: ", sum(1 for _ in doc if _.pos_ == "VERB"))
  print("Sustantivos :", [chunk.text for chunk in doc.noun_chunks])
  print("Verbos :", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Buscamos entidades, frases y conceptos
  for entity in doc.ents:
      print(entity.text, entity.label_)

  tiempo_total = datetime.now() - tiempo_total

  print( "Tiempo de renocimiento de palabras:", tiempo_en_ms(tiempo_text) , "[ms]" )
  print( "Tiempo Total:",                   tiempo_en_ms(tiempo_total) , "[ms]" )

Cantidad de Sustantivos:  749
Cantidad de Verbos:  472
Sustantivos : ['It', 'the middle', 'winter', 'the snow-flakes', 'feathers', 'the sky', 'a queen', 'her window', 'her embroidery-frame', 'ebony', 'she', 'times', 'the snow', 'she', 'her finger', 'it', 'three drops', 'blood', 'the snow', 'she', 'it', 'she', 'herself', 'I', 'a child', 'snow', 'blood', 'the wood', 'the embroidery frame', 'she', 'a daughter', 'a skin', 'snow', 'blood', 'hair', 'ebony', 'she', 'she', 'the queen', 'a year', 'the king', 'another wife', 'a beautiful woman', 'she', 'beauty', 'any one', 'She', 'a magic looking-glass', 'she', 'it', 'it', 'the wall', 'Who', 'us', 'the looking-glass', 'You', 'them', 'she', 'she', 'the looking-glass', 'the truth', 'Snow-white', 'she', 'she', 'day', 'the queen', 'herself', 'the queen', 'her mirror', '"Looking-glass', 'the wall', 'Who', 'us', 'It', 'you', 'you', 'the queen', 'a great shock', 'she', 'envy', 'that hour', 'her heart', 'she', 'her', 'envy', 'pride', 'ill weeds', 'her h

# Desarrollo en GPU

In [3]:
import spacy
from datetime import datetime
import matplotlib.pyplot as plt


# --------------------------------------------
# Definición de función que transforma el tiempo en  milisegundos 
tiempo_en_ms = lambda dt:(dt.days * 24 * 60 * 60 + dt.seconds) * 1000 + dt.microseconds / 1000.0
# --------------------------------------------
spacy.prefer_gpu()


texto_a_analizar="It was the middle of winter, and the snow-flakes were falling like feathers from the sky, and a queen sat at her window working, and her embroidery-frame was of ebony. And as she worked, gazing at times out on the snow, she pricked her finger, and there fell from it three drops of blood on the snow. And when she saw how bright and red it looked, she said to herself, \"Oh that I had a child as white as snow, as red as blood, and as black as the wood of the embroidery frame!\" Not very long after she had a daughter, with a skin as white as snow, lips as red as blood, and hair as black as ebony, and she was named Snow-white. And when she was born the queen died. After a year had gone by the king took another wife, a beautiful woman, but proud and overbearing, and she could not bear to be surpassed in beauty by any one. She had a magic looking-glass, and she used to stand before it, and look in it, and say,  \"Looking-glass upon the wall, Who is fairest of us all?\"  And the looking-glass would answer,  \"You are fairest of them all.\"  And she was contented, for she knew that the looking-glass spoke the truth. Now, Snow-white was growing prettier and prettier, and when she was seven years old she was as beautiful as day, far more so than the queen herself. So one day when the queen went to her mirror and said,  \"Looking-glass upon the wall, Who is fairest of us all?\"  It answered,  \"Queen, you are full fair, 'tis true, But Snow-white fairer is than you.\"  This gave the queen a great shock, and she became yellow and green with envy, and from that hour her heart turned against Snow-white, and she hated her. And envy and pride like ill weeds grew in her heart higher every day, until she had no peace day or night. At last she sent for a huntsman, and said, \"Take the child out into the woods, so that I may set eyes on her no more. You must put her to death, and bring me her heart for a token.\" The huntsman consented, and led her away; but when he drew his cutlass to pierce Snow-white's innocent heart, she began to weep, and to say, \"Oh, dear huntsman, do not take my life; I will go away into the wild wood, and never come home again.\" And as she was so lovely the huntsman had pity on her, and said, \"Away with you then, poor child;\" for he thought the wild animals would be sure to devour her, and it was as if a stone had been rolled away from his heart when he spared to put her to death. Just at that moment a young wild boar came running by, so he caught and killed it, and taking out its heart, he brought it to the queen for a token. And it was salted and cooked, and the wicked woman ate it up, thinking that there was an end of Snow-white.  Now, when the poor child found herself quite alone in the wild woods, she felt full of terror, even of the very leaves on the trees, and she did not know what to do for fright. Then she began to run over the sharp stones and through the thorn bushes, and the wild beasts after her, but they did her no harm. She ran as long as her feet would carry her; and when the evening drew near she came to a little house, and she went inside to rest. Everything there was very small, but as pretty and clean as possible. There stood the little table ready laid, and covered with a white cloth, and seven little plates, and seven knives and forks, and drinking-cups. By the wall stood seven little beds, side by side, covered with clean white quilts. Snow-white, being very hungry and thirsty, ate from each plate a little porridge and bread, and drank out of each little cup a drop of wine, so as not to finish up one portion alone. After that she felt so tired that she lay down on one of the beds, but it did not seem to suit her; one was too long, another too short, but at last the seventh was quite right; and so she lay down upon it, committed herself to heaven, and fell asleep.  When it was quite dark, the masters of the house came home. They were seven dwarfs, whose occupation was to dig underground among the mountains. When they had lighted their seven candles, and it was quite light in the little house, they saw that some one must have been in, as everything was not in the same order in which they left it. The first said, \"Who has been sitting in my little chair?\" The second said, \"Who has been eating from my little plate?\" The third said, \"Who has been taking my little loaf?\" The fourth said, \"Who has been tasting my porridge?\" The fifth said, \"Who has been using my little fork?\" The sixth said, \"Who has been cutting with my little knife?\" The seventh said, \"Who has been drinking from my little cup?\" Then the first one, looking round, saw a hollow in his bed, and cried, \"Who has been lying on my bed?\" And the others came running, and cried, \"Some one has been on our beds too!\" But when the seventh looked at his bed, he saw little Snow-white lying there asleep. Then he told the others, who came running up, crying out in their astonishment, and holding up their seven little candles to throw a light upon Snow-white. \"O goodness! O gracious!\" cried they, \"what beautiful child is this?\" and were so full of joy to see her that they did not wake her, but let her sleep on. And the seventh dwarf slept with his comrades, an hour at a time with each, until the night had passed. When it was morning, and Snow-white awoke and saw the seven dwarfs, she was very frightened; but they seemed quite friendly, and asked her what her name was, and she told them; and then they asked how she came to be in their house. And she related to them how her step-mother had wished her to be put to death, and how the huntsman had spared her life, and how she had run the whole day long, until at last she had found their little house. Then the dwarfs said, \"If you will keep our house for us, and cook, and wash, and make the beds, and sew and knit, and keep everything tidy and clean, you may stay with us, and you shall lack nothing.\" - \"With all my heart,\" said Snow-white; and so she stayed, and kept the house in good order. In the morning the dwarfs went to the mountain to dig for gold; in the evening they came home, and their supper had to be ready for them. All the day long the maiden was left alone, and the good little dwarfs warned her, saying, \"Beware of your step-mother, she will soon know you are here. Let no one into the house.\" Now the queen, having eaten Snow-white's heart, as she supposed, felt quite sure that now she was the first and fairest, and so she came to her mirror, and said,  \"Looking-glass upon the wall, Who is fairest of us all?\"  And the glass answered,  \"Queen, thou art of beauty rare, But Snow-white living in the glen With the seven little men Is a thousand times more fair.\"  Then she was very angry, for the glass always spoke the truth, and she knew that the huntsman must have deceived her, and that Snow-white must still be living. And she thought and thought how she could manage to make an end of her, for as long as she was not the fairest in the land, envy left her no rest. At last she thought of a plan; she painted her face and dressed herself like an old pedlar woman, so that no one would have known her. In this disguise she went across the seven mountains, until she came to the house of the seven little dwarfs, and she knocked at the door and cried, \"Fine wares to sell! fine wares to sell!\" Snow-white peeped out of the window and cried, \"Good-day, good woman, what have you to sell?\" - \"Good wares, fine wares,\" answered she, \"laces of all colours;\"and she held up a piece that was woven of variegated silk. \"I need not be afraid of letting in this good woman,\" thought Snow-white, and she unbarred the door and bought the pretty lace. \"What a figure you are, child!\" said the old woman, \"come and let me lace you properly for once.\" Snow-white, suspecting nothing, stood up before her, and let her lace her with the new lace; but the old woman laced so quick and tight that it took Snow-white's breath away, and she fell down as dead. \"Now you have done with being the fairest,\" said the old woman as she hastened away. Not long after that, towards evening, the seven dwarfs came home, and were terrified to see their dear Snow-white lying on the ground, without life or motion; they raised her up, and when they saw how tightly she was laced they cut the lace in two; then she began to draw breath, and little by little she returned to life. When the dwarfs heard what had happened they said, \"The old pedlar woman was no other than the wicked queen; you must beware of letting any one in when we are not here!\" And when the wicked woman got home she went to her glass and said,  \"Looking-glass against the wall, Who is fairest of us all?\"  And it answered as before,  \"Queen, thou art of beauty rare, But Snow-white living in the glen With the seven little men Is a thousand times more fair.\"  When she heard that she was so struck with surprise that all the blood left her heart, for she knew that Snow-white must still be living. \"But now,\" said she, \"I will think of something that will be her ruin.\" And by witchcraft she made a poisoned comb. Then she dressed herself up to look like another different sort of old woman. So she went across the seven mountains and came to the house of the seven dwarfs, and knocked at the door and cried, \"Good wares to sell! good wares to sell!\" Snow-white looked out and said, \"Go away, I must not let anybody in.\" - \"But you are not forbidden to look,\" said the old woman, taking out the poisoned comb and holding it up. It pleased the poor child so much that she was tempted to open the door; and when the bargain was made the old woman said, \"Now, for once your hair shall be properly combed.\" Poor Snow-white, thinking no harm, let the old woman do as she would, but no sooner was the comb put in her hair than the poison began to work, and the poor girl fell down senseless. \"Now, you paragon of beauty,\" said the wicked woman, \"this is the end of you,\" and went off. By good luck it was now near evening, and the seven little dwarfs came home. When they saw Snow-white lying on the ground as dead, they thought directly that it was the step-mother's doing, and looked about, found the poisoned comb, and no sooner had they drawn it out of her hair than Snow-white came to herself, and related all that had passed. Then they warned her once more to be on her guard, and never again to let any one in at the door. And the queen went home and stood before the looking-glass and said,  \"Looking-glass against the wall, Who is fairest of us all?\"  And the looking-glass answered as before,  \"Queen, thou art of beauty rare, But Snow-white living in the glen With the seven little men Is a thousand times more fair.\"  When she heard the looking-glass speak thus she trembled and shook with anger. \"Snow-white shall die,\" cried she, \"though it should cost me my own life!\" And then she went to a secret lonely chamber, where no one was likely to come, and there she made a poisonous apple. It was beautiful to look upon, being white with red cheeks, so that any one who should see it must long for it, but whoever ate even a little bit of it must die. When the apple was ready she painted her face and clothed herself like a peasant woman, and went across the seven mountains to where the seven dwarfs lived. And when she knocked at the door Snow-white put her head out of the window and said, \"I dare not let anybody in; the seven dwarfs told me not.\" - \"All right,\" answered the woman; \"I can easily get rid of my apples elsewhere. There, I will give you one.\" - \"No,\" answered Snow-white, \"I dare not take anything.\" - \"Are you afraid of poison?\" said the woman, \"look here, I will cut the apple in two pieces; you shall have the red side, I will have the white one.\" For the apple was so cunningly made, that all the poison was in the rosy half of it. Snow-white longed for the beautiful apple, and as she saw the peasant woman eating a piece of it she could no longer refrain, but stretched out her hand and took the poisoned half. But no sooner had she taken a morsel of it into her mouth than she fell to the earth as dead. And the queen, casting on her a terrible glance, laughed aloud and cried, \"As white as snow, as red as blood, as black as ebony! this time the dwarfs will not be able to bring you to life again.\" And when she went home and asked the looking-glass,  \"Looking-glass against the wall, Who is fairest of us all?\"  at last it answered,  \"You are the fairest now of all.\"  Then her envious heart had peace, as much as an envious heart can have. The dwarfs, when they came home in the evening, found Snow-white lying on the ground, and there came no breath out of her mouth, and she was dead. They lifted her up, sought if anything poisonous was to be found, cut her laces, combed her hair, washed her with water and wine, but all was of no avail, the poor child was dead, and remained dead. Then they laid her on a bier, and sat all seven of them round it, and wept and lamented three whole days. And then they would have buried her, but that she looked still as if she were living, with her beautiful blooming cheeks. So they said, \"We cannot hide her away in the black ground.\" And they had made a coffin of clear glass, so as to be looked into from all sides, and they laid her in it, and wrote in golden letters upon it her name, and that she was a king's daughter. Then they set the coffin out upon the mountain, and one of them always remained by it to watch. And the birds came too, and mourned for Snow-white, first an owl, then a raven, and lastly, a dove. Now, for a long while Snow-white lay in the coffin and never changed, but looked as if she were asleep, for she was still as' white as snow, as red as blood, and her hair was as black as ebony. It happened, however, that one day a king's son rode through the wood and up to the dwarfs' house, which was near it. He saw on the mountain the coffin, and beautiful Snow-white within it, and he read what was written in golden letters upon it. Then he said to the dwarfs, \"Let me have the coffin, and I will give you whatever you like to ask for it.\" But the dwarfs told him that they could not part with it for all the gold in the world. But he said, \"I beseech you to give it me, for I cannot live without looking upon Snow-white; if you consent I will bring you to great honour, and care for you as if you were my brethren.\" When he so spoke the good little dwarfs had pity upon him and gave him the coffin, and the king's son called his servants and bid them carry it away on their shoulders. Now it happened that as they were going along they stumbled over a bush, and with the shaking the bit of poisoned apple flew out of her throat. It was not long before she opened her eyes, threw up the cover of the coffin, and sat up, alive and well. \"Oh dear! where am I?\" cried she. The king's son answered, full of joy, \"You are near me,\" and, relating all that had happened, he said, \"I would rather have you than anything in the world; come with me to my father's castle and you shall be my bride.\" And Snow-white was kind, and went with him, and their wedding was held with pomp and great splendour. But Snow-white's wicked step-mother was also bidden to the feast, and when she had dressed herself in beautiful clothes she went to her looking-glass and said,  \"Looking-glass upon the wall, Who is fairest of us all?\"  The looking-glass answered,  ''O Queen, although you are of beauty rare, The young bride is a thousand times more fair.\"" #@param {type:"string"}
idioma = "Ingles" #@param ["Aleman", "Espanol", "Ingles"]

idioma_seleccionado ={

'Aleman' : "de_core_web_sm",

'Espanol' : "es_core_news_sm",

'Ingles' : "en_core_web_sm",

}

if texto_a_analizar == "":
  print("No se ingreso ningun texto a analizar.")
else:
  tiempo_total = datetime.now()
# Se tokeniza el paquete de idioma seleccionado.
  nlp = spacy.load(idioma_seleccionado.get(idioma,-1))

# Procesamos el texto ingresado
  tiempo_text = datetime.now()
  doc = nlp(texto_a_analizar)
  tiempo_text = datetime.now() - tiempo_text

# Analizamos la sintaxis
  print("Cantidad de Sustantivos: ", sum(1 for _ in doc.noun_chunks))
  print("Cantidad de Verbos: ", sum(1 for _ in doc if _.pos_ == "VERB"))
  print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
  print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Buscamos entidades, frases y conceptos
  for entity in doc.ents:
      print(entity.text, entity.label_)


  tiempo_total = datetime.now() - tiempo_total

  print( "Tiempo de conversión de imagen:", tiempo_en_ms(tiempo_text)   , "[ms]" )
  print( "Tiempo Total:",                   tiempo_en_ms(tiempo_total) , "[ms]" )

Cantidad de Sustantivos:  749
Cantidad de Verbos:  472
Noun phrases: ['It', 'the middle', 'winter', 'the snow-flakes', 'feathers', 'the sky', 'a queen', 'her window', 'her embroidery-frame', 'ebony', 'she', 'times', 'the snow', 'she', 'her finger', 'it', 'three drops', 'blood', 'the snow', 'she', 'it', 'she', 'herself', 'I', 'a child', 'snow', 'blood', 'the wood', 'the embroidery frame', 'she', 'a daughter', 'a skin', 'snow', 'blood', 'hair', 'ebony', 'she', 'she', 'the queen', 'a year', 'the king', 'another wife', 'a beautiful woman', 'she', 'beauty', 'any one', 'She', 'a magic looking-glass', 'she', 'it', 'it', 'the wall', 'Who', 'us', 'the looking-glass', 'You', 'them', 'she', 'she', 'the looking-glass', 'the truth', 'Snow-white', 'she', 'she', 'day', 'the queen', 'herself', 'the queen', 'her mirror', '"Looking-glass', 'the wall', 'Who', 'us', 'It', 'you', 'you', 'the queen', 'a great shock', 'she', 'envy', 'that hour', 'her heart', 'she', 'her', 'envy', 'pride', 'ill weeds', 'her h

# Conclusión



Luego de haber hecho algunas pruebas con diferentes sets de datos variando el tamaño de los mismos, pudimos observar que a medida que varia el volumen del texto ingresado, el tiempo en analizar el texto entre cpu y gpu varia siendo el analisis del gpu un poco mas eficiente que el de cpu. De todas formas, el analisis utilizando cpu es muy bueno debido a la optimizacion que posee esta biblioteca.

# Bibliografia



[1] SpaCy: [Documentación spaCy](https://spacy.io/)

[2] Fast and Accurate Entity Recognition with Iterated Dilated Convolutions: [Paper](https://arxiv.org/abs/1702.02098) 

[3] A Replicable Comparison Study of NER Software: StanfordNLP, NLTK, OpenNLP, SpaCy, Gate: [Paper](https://www.researchgate.net/publication/337977695_A_Replicable_Comparison_Study_of_NER_Software_StanfordNLP_NLTK_OpenNLP_SpaCy_Gate)


